In [ ]:
import torch
from datasets import load_dataset, DatasetDict
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

from datasets import load_dataset

# dataset link: https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset
data = load_dataset(
    'csv',
    data_files={
        'train': r'D:\AU\Internships\DHC - ML\AI_ML_AdvancedTasks\AI-ML-AdvancedTask1_HussainAbdullah\ag_news\train.csv',
        'test': r'D:\AU\Internships\DHC - ML\AI_ML_AdvancedTasks\AI-ML-AdvancedTask1_HussainAbdullah\ag_news\test.csv'
    }
)
data = data.rename_column('description', 'text').rename_column('label', 'labels')

# Tokenizer & tokenize
model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)

data = data.map(preprocess, batched=True)
data = data.remove_columns(['text','title'])  # drop unused
data.set_format('torch')

# Model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

# Training
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('./agnews-bert-classifier')
tokenizer.save_pretrained('./agnews-bert-classifier')